# 📖 PDF Book OCR with Marker

This notebook extracts text and images from PDF books using **Marker** (the best open-source OCR library in 2025).

**Features:**
- 🚀 GPU-accelerated processing
- 📝 Outputs structured Markdown
- 🖼️ Extracts images from PDFs
- 📊 Preserves tables and equations

**Instructions:**
1. Run cells in order
2. Upload your PDF when prompted
3. Download the `.md` and images when done

## Step 1: Install Dependencies

In [ ]:
# Install Marker (takes ~2-3 minutes)
!pip install marker-pdf -q
print("✅ Marker installed!")

## Step 2: Check GPU Availability

In [ ]:
import torch

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU Available: {gpu_name}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected. Processing will be slower.")
    print("   Go to Runtime > Change runtime type > GPU")

## Step 3: Upload Your PDF

**Option A:** Upload directly from your computer  
**Option B:** Mount Google Drive and use a PDF from there

In [ ]:
# Choose your upload method
UPLOAD_METHOD = "direct"  # Change to "drive" to use Google Drive

if UPLOAD_METHOD == "direct":
    from google.colab import files
    print("📁 Select your PDF file to upload...")
    uploaded = files.upload()
    PDF_PATH = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {PDF_PATH}")
else:
    from google.colab import drive
    drive.mount('/content/drive')
    # Set your PDF path in Drive
    PDF_PATH = "/content/drive/MyDrive/Your_Book.pdf"  # ⬅️ EDIT THIS
    print(f"✅ Using: {PDF_PATH}")

## Step 4: Extract Text and Images from PDF

In [ ]:
import os
import time
from pathlib import Path

from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

pdf_path = Path(PDF_PATH)
output_name = pdf_path.stem

print(f"\n📖 Processing: {pdf_path.name}")
print(f"   Size: {pdf_path.stat().st_size / 1024 / 1024:.1f} MB")
print("-" * 50)

start_time = time.time()

# Load models (first run downloads ~3GB of models)
print("🔧 Loading OCR models (first run downloads ~3GB)...")
model_dict = create_model_dict()
converter = PdfConverter(artifact_dict=model_dict)

# Convert PDF
print("📝 Extracting text and images...")
rendered = converter(str(pdf_path))

# Get text and images
text, _, images = text_from_rendered(rendered)

elapsed = time.time() - start_time
print(f"\n✅ Extraction complete!")
print(f"   Time: {elapsed:.1f} seconds")
print(f"   Text length: {len(text):,} characters")
print(f"   Images found: {len(images) if images else 0}")

## Step 5: Save Output Files

In [ ]:
# Create output directory
output_dir = Path(f"/content/{output_name}_output")
output_dir.mkdir(exist_ok=True)

# Save Markdown
md_path = output_dir / f"{output_name}.md"
md_path.write_text(text, encoding="utf-8")
print(f"✅ Markdown saved: {md_path}")

# Save images
if images:
    images_dir = output_dir / "images"
    images_dir.mkdir(exist_ok=True)
    
    for img_name, img_data in images.items():
        img_path = images_dir / img_name
        if hasattr(img_data, 'save'):
            img_data.save(str(img_path))
        elif isinstance(img_data, bytes):
            img_path.write_bytes(img_data)
    
    print(f"✅ Images saved: {len(images)} files in {images_dir}")

print(f"\n📁 All files saved to: {output_dir}")

## Step 6: Preview Output

In [ ]:
# Preview first 2000 characters
print("=" * 60)
print("📄 PREVIEW (first 2000 characters)")
print("=" * 60)
print(text[:2000])
if len(text) > 2000:
    print("\n... [truncated] ...")

## Step 7: Download Output

Run this cell to download the extracted Markdown and images as a ZIP file.

In [ ]:
import shutil
from google.colab import files

# Create ZIP file
zip_path = f"/content/{output_name}_extracted"
shutil.make_archive(zip_path, 'zip', output_dir)

print(f"📦 Created: {zip_path}.zip")
print("\n📥 Starting download...")

# Download
files.download(f"{zip_path}.zip")

print("\n🎉 Done! Upload the .md file to GraphRecall to ingest it.")

---

## 💡 Tips

1. **Speed**: With Colab GPU (T4), a 200-page book takes ~3-5 minutes
2. **Large PDFs**: If you hit memory limits, try a smaller PDF or use Colab Pro
3. **Next Steps**: Upload the `.md` file to GraphRecall for concept extraction and quiz generation